#Unscented Kalman Filter (additive noise) Example

###Introduction

This notebook is designed to demonstrate how to use the StateSpace.jl package to execute the Unscented Kalman filter with additive noise for a non-linear State Space model. The example that has been used here closely follows the example given in a paper by Rambabu Kandepu, Bjarne Foss and Lars Imsland ["Applying the unscented Kalman filter for nonlinear state estimation".](http://folk.ntnu.no/bjarnean/pubs/journals/journal-47.pdf) The example is the [Van der Pol oscillator](https://en.wikipedia.org/wiki/Van_der_Pol_oscillator) (Section 4.1 on page 6 of the paper). You can read the paper for full details.   

For those of you that do not need/want the explanation of the model and the code, you can skip right to the end of this notebook where the entire section of code required to run this example is given.

### The Problem

The problem that we will consider here is that of predicting the position $\mathbf{x} = (x_1, x_2)$ of a Van der Pol oscillator given some noisy measurements of its position.

####The Process Model
The dynamic equations describing the rate of change of position of the oscillator are:

$$
\begin{align}
\frac{\mathrm{d}x_1}{\mathrm{d}t} &= -x_2, \\
\frac{\mathrm{d}x_2}{\mathrm{d}t} &= -\mu \left(1 - x_1^2 \right)x_2 + x_1, \\
\end{align}
$$

where $\mu$ is a scalar parameter that defines the strength of the damping.   
These equations can be written in a discrete manner required for solution with the unscented Kalman filter as follows: 

$$
\begin{align}
x_1^n &= x_1^{n-1} - \Delta t \times x_2^{n-1}, \\
x_2^n &= x_2^{n-1} - \Delta t \times \mu \left(1 - (x_1^{n-1})^2 \right)x_2^{n-1} + x_1^{n-1}, \\
\end{align}
$$

where $\mathbf{x}^n = \left( x_1^n, x_2^n \right)$ represents the position of the oscillator at time step $n$ and $\Delta t$ is the time step between consecutive measurements.

#### The Observation Model
We assume that we measure the position of the oscillator directly and hence the observation function is simply
$$
y^n(\mathbf{x}^n) = \left( x_1^n, x_2^n \right) 
$$

###Setting up the problem
First we'll import the required modules

In [1]:
using StateSpace
using Distributions
using Gadfly
using DataFrames
using Colors

#####Define Kalman Filter Parameters
We now need to define the process and observation model according to the model described above.

In [2]:
Δt = 0.1 # Set the time step

#Set the process function
function processFunction(x::Vector, Δt::Float64=0.1, μ::Float64=0.3)
    x1 = zeros(x)
    x1[1] = x[1] + Δt * -x[2]
    x1[2] = x[2] + Δt * (-μ * (1 - x[1]^2) * x[2] + x[1])
    x1
end

#Set the observation function.
observationFunction(x::Vector) = x

#Set process noise covariance matrix
processCovariance = 1e-2*[0.1 0.0; 0.0 1e-3]

#Set the observation noise covariance
observationCovariance = 1e-1*eye(2)

#Create additive noise UKF model
ukfStateModel = AdditiveNonLinUKFSSM(processFunction, processCovariance, observationFunction, observationCovariance)

StateSpace.AdditiveNonLinUKFSSM{Float64}(processFunction,2x2 Array{Float64,2}:
 0.001  0.0   
 0.0    1.0e-5,observationFunction,2x2 Array{Float64,2}:
 0.1  0.0
 0.0  0.1)

#####Generate noisy observations
Here we'll generate the noisy observations that will be passed to the Unscented Kalman Filter.

First we define the true initial position of the system

In [3]:
trueInitialState = [1.4, 0.0] #Set the true initial state of the oscillator

2-element Array{Float64,1}:
 1.4
 0.0

Now we can define the number of observations and generate the true and noisy observations

In [4]:
#Set the number of observations and generate the true values and the noisy
#observations.
numObs = 200
trueState = zeros(2,numObs)
noisyObs = zeros(2,numObs)
trueState[:,1] = trueInitialState
noisyObs[:,1] = trueInitialState + sqrt(observationCovariance)*randn(2)
for i in 2:numObs
    trueState[:,i] = processFunction(trueState[:,i-1])
    noisyObs[:,i] = trueState[:,i] + sqrt(observationCovariance)*randn(2)
end

####Guess of initial state
Now we define the guess of the initial state of the system

In [5]:
initial_guess = MvNormal([0.0,5.0], 5.0*eye(2))

FullNormal(
dim: 2
μ: [0.0,5.0]
Σ: 2x2 Array{Float64,2}:
 5.0  0.0
 0.0  5.0
)


#####Perform Unscented Kalman Filter (additive noise) Algorithm
Now we have all of the parameters:
1. noisy observations
2. process (transition) and observation (emission) model paramaters
3. initial guess of state   

We can use the Unscented Kalman Filter to predict the true underlying state (position of the oscillator).

In [6]:
filtered_state = filter(ukfStateModel, noisyObs, initial_guess)

SmoothedState{Float64}


200 estimates of 2-D process from 2-D observations
Log-likelihood: 0.0


NOTE: The log likelihood hasn't been implemented for the Unscented Kalman Filter and so 0.0 is just the default value set. It doesn't mean anything yet. We will sort this out soon. 

###Plot the results
Now we will plot the results of the $x_1$ position of the oscillator.

First we extract the mean $x_1$ positions and their variances from the filtered estimates. 

In [7]:
x_data = 1:numObs
x1_array = Vector{Float64}(numObs)
x1Var_array = Vector{Float64}(numObs)
x1_Guess = initial_guess.μ[1]
x1Var_Guess = 2*sqrt(initial_guess.Σ.mat[1,1])
for i in x_data
    current_state = filtered_state.state[i]
    x1_array[i] = current_state.μ[1]
    x1Var_array[i] = 2*sqrt(current_state.Σ.mat[1,1])
end

Next we will create a dataframe. This is simply so the syntax is simple for plotting the ribbon digram which will represent the state along with the 95% confidence interval

In [8]:
df_fs = DataFrame(
    x = x_data*Δt,
    y = x1_array,
    ymin = x1_array - x1Var_array,
    ymax = x1_array + x1Var_array,
    f = "Filtered values"
    )

,x,y,ymin,ymax,f
1,0.1,1.2311443520702978,0.601673977105222,1.8606147270353737,Filtered values
2,0.2,1.207639574274799,0.7559528199182741,1.6593263286313238,Filtered values
3,0.30000000000000004,1.2404706031820314,0.8665527972788076,1.6143884090852552,Filtered values
4,0.4,1.2624521557504829,0.9336425977767057,1.59126171372426,Filtered values
5,0.5,1.3021074556781196,1.0031239121655415,1.6010909991906976,Filtered values
6,0.6,1.1439791942686566,0.8663001404290013,1.421658248108312,Filtered values
7,0.7000000000000001,1.05869138516419,0.7968256010214874,1.3205571693068925,Filtered values
8,0.8,1.02839634512671,0.7785803362354985,1.2782123540179215,Filtered values
9,0.9,0.9334672736021697,0.6930219174350944,1.173912629769245,Filtered values
10,1.0,0.8942465135716597,0.661068482363518,1.1274245447798015,Filtered values


Next we create 3 distinguishable colors for the plot

In [9]:
n = 3
getColors = distinguishable_colors(n, Color[LCHab(70, 60, 240)],
                                   transform=c -> deuteranopic(c, 0.5),
                                   lchoices=Float64[65, 70, 75, 80],
                                   cchoices=Float64[0, 50, 60, 70],
                                   hchoices=linspace(0, 330, 24))

Finally we display the plot

In [10]:
oscillatorx1_state_plot = plot(
    layer(x=x_data*Δt, y=noisyObs[1,:], Geom.point, Theme(default_color=getColors[2])),
    layer(x=x_data*Δt, y=trueState[1,:], Geom.line, Theme(default_color=getColors[3])),
    layer(df_fs, x=:x, y=:y, ymin=:ymin, ymax=:ymax, Geom.line, Geom.ribbon),
    Guide.xlabel("Time (seconds)"), Guide.ylabel("x1"),
    Guide.manual_color_key("Colour Key",["Filtered Estimate", "Measurements","True Value "],[getColors[1],getColors[2],getColors[3]]),
    Guide.title("Unscented Kalman Filter (Additive) Example")
    )
display(oscillatorx1_state_plot)

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 Time (seconds) 
 
 
 -30 
 -25 
 -20 
 -15 
 -10 
 -5 
 0 
 5 
 10 
 15 
 20 
 25 
 30 
 35 
 40 
 45 
 50 
 55 
 -25 
 -24 
 -23 
 -22 
 -21 
 -20 
 -19 
 -18 
 -17 
 -16 
 -15 
 -14 
 -13 
 -12 
 -11 
 -10 
 -9 
 -8 
 -7 
 -6 
 -5 
 -4 
 -3 
 -2 
 -1 
 0 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 16 
 17 
 18 
 19 
 20 
 21 
 22 
 23 
 24 
 25 
 26 
 27 
 28 
 29 
 30 
 31 
 32 
 33 
 34 
 35 
 36 
 37 
 38 
 39 
 40 
 41 
 42 
 43 
 44 
 45 
 46 
 47 
 48 
 49 
 50 
 -25 
 0 
 25 
 50 
 -26 
 -24 
 -22 
 -20 
 -18 
 -16 
 -14 
 -12 
 -10 
 -8 
 -6 
 -4 
 -2 
 0 
 2 
 4 
 6 
 8 
 10 
 12 
 14 
 16 
 18 
 20 
 22 
 24 
 26 
 28 
 30 
 32 
 34 
 36 
 38 
 40 
 42 
 44 
 46 
 48 
 50 
 
 
 
 Filtered Estimate 
 Measurements 
 True Value 
 
 
 
 
 
 
 
 Colour Key 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M91.89,47.77 L 91.52 47.89 91.16 47.07 90.8 45.72 90.44 44.82 90.07 43.83 89.71 42.34 89.35 41.7 88.99 41.08 88.62 41.61 88.26 41.94 87.9 41.28 87.54 41.52 87.17 43 86.81 43.44 86.45 43.6 86.09 44.62 85.72 45.06 85.36 46.23 85 46.01 84.64 46.37 84.27 47.14 83.91 47.61 83.55 48.35 83.19 48.53 82.83 49.06 82.46 49.8 82.1 51.04 81.74 51.38 81.38 51.64 81.01 51.21 80.65 51.99 80.29 52.17 79.93 52.65 79.56 53.41 79.2 54.15 78.84 54.32 78.48 55.63 78.11 56.05 77.75 56.82 77.39 57.68 77.03 57.29 76.66 57.73 76.3 57.75 75.94 58.18 75.58 58.41 75.21 58.33 74.85 57.83 74.49 57.25 74.13 57.33 73.76 57.03 73.4 57.02 73.04 56.02 72.68 54.88 72.31 53.55 71.95 53.01 71.59 52.45 71.23 51.99 70.86 50.63 70.5 50 70.14 48.93 69.78 47.76 69.41 47.26 69.05 46.26 68.69 45.51 68.33 44.72 67.97 43.56 67.6 42.96 67.24 41.23 66.88 40.65 66.52 40.6 66.15 40.39 65.79 39.59 65.43 39.22 65.07 38.75 64.7 37.59 64.34 37.88 63.98 37.96 63.62 38.37 63.25 38.54 62.89 38.73 62.53 39.28 62.17 39.52 61.8 41.13 61.44 42.91 61.08 43.5 60.72 44.43 60.35 45.72 59.99 47.05 59.63 48.34 59.27 49.45 58.9 50.47 58.54 51.94 58.18 53.31 57.82 54.48 57.45 56.49 57.09 57.24 56.73 58.84 56.37 60.18 56 61.2 55.64 62.39 55.28 63.74 54.92 64.63 54.55 65.45 54.19 66.51 53.83 66.76 53.47 67.3 53.11 67.31 52.74 66.98 52.38 65.95 52.02 64.52 51.66 63.95 51.29 63.13 50.93 62.53 50.57 61.68 50.21 60.82 49.84 58.81 49.48 56.85 49.12 55.27 48.76 54.29 48.39 52.82 48.03 51.06 47.67 49.04 47.31 47.56 46.94 45.01 46.58 42.14 46.22 39.8 45.86 38.76 45.49 36.63 45.13 34.81 44.77 33.53 44.41 32.67 44.04 31.73 43.68 30.7 43.32 30.11 42.96 29.96 42.59 29.88 42.23 30.03 41.87 30.59 41.51 31.45 41.14 32.86 40.78 34.8 40.42 34.86 40.06 35.26 39.69 36.04 39.33 37.65 38.97 37.91 38.61 40.27 38.25 41.57 37.88 43.23 37.52 45.33 37.16 47.81 36.8 48.93 36.43 52.27 36.07 54.99 35.71 57.42 35.35 59.35 34.98 61.5 34.62 63.6 34.26 64.44 33.9 65.51 33.53 67.29 33.17 67.96 32.81 68.36 32.45 69.59 32.08 69.76 31.72 69.84 31.36 69.98 31 69.53 30.63 68.96 30.27 68.88 29.91 68.06 29.55 66.98 29.18 65.87 28.82 65.32 28.46 64.41 28.1 63.27 27.73 61.69 27.37 59.54 27.01 58.35 26.65 55.68 26.28 53.65 25.92 51.34 25.56 48.78 25.2 46.84 24.83 45.12 24.47 43.94 24.11 42.08 23.75 39.54 23.39 38.21 23.02 36.07 22.66 35.56 22.3 34.19 21.94 33.89 21.57 32.78 21.21 30.59 20.85 31.7 20.49 32.78 20.12 34.55 19.76 37.02 19.76 16.85 20.12 20.07 20.49 20.79 20.85 21.16 21.21 21 21.57 23.88 21.94 25.5 22.3 26.18 22.66 27.85 23.02 28.6 23.39 30.92 23.75 32.38 24.11 35.02 24.47 36.95 24.83 38.18 25.2 39.95 25.56 41.93 25.92 44.53 26.28 46.87 26.65 48.93 27.01 51.63 27.37 52.85 27.73 55.02 28.1 56

For those that just want the code, without the explanation, you have come to the right place. Here it is:

In [11]:
using StateSpace
using Distributions
using Gadfly
using DataFrames
using Colors

Δt = 0.1 # Set the time step

#Set the process function
function processFunction(x::Vector, Δt::Float64=0.1, μ::Float64=0.3)
    x1 = zeros(x)
    x1[1] = x[1] + Δt * -x[2]
    x1[2] = x[2] + Δt * (-μ * (1 - x[1]^2) * x[2] + x[1])
    x1
end

#Set the observation function.
observationFunction(x::Vector) = x

#Set process noise covariance matrix
processCovariance = 1e-2*[0.1 0.0; 0.0 1e-3]

#Set the observation noise covariance
observationCovariance = 1e-1*eye(2)

#Create additive noise UKF model
ukfStateModel = AdditiveNonLinUKFSSM(processFunction, processCovariance, observationFunction, observationCovariance)

trueInitialState = [1.4, 0.0] #Set the true initial state of the oscillator

#Set the number of observations and generate the true values and the noisy
#observations.
numObs = 200
trueState = zeros(2,numObs)
noisyObs = zeros(2,numObs)
trueState[:,1] = trueInitialState
noisyObs[:,1] = trueInitialState + sqrt(observationCovariance)*randn(2)
for i in 2:numObs
    trueState[:,i] = processFunction(trueState[:,i-1])
    noisyObs[:,i] = trueState[:,i] + sqrt(observationCovariance)*randn(2)
end

initial_guess = MvNormal([0.0,5.0], 5.0*eye(2))

filtered_state = filter(ukfStateModel, noisyObs, initial_guess)

x_data = 1:numObs
x1_array = Vector{Float64}(numObs)
x1Var_array = Vector{Float64}(numObs)
x1_Guess = initial_guess.μ[1]
x1Var_Guess = 2*sqrt(initial_guess.Σ.mat[1,1])
for i in x_data
    current_state = filtered_state.state[i]
    x1_array[i] = current_state.μ[1]
    x1Var_array[i] = 2*sqrt(current_state.Σ.mat[1,1])
end

df_fs = DataFrame(
    x = x_data*Δt,
    y = x1_array,
    ymin = x1_array - x1Var_array,
    ymax = x1_array + x1Var_array,
    f = "Filtered values"
    )

n = 3
getColors = distinguishable_colors(n, Color[LCHab(70, 60, 240)],
                                   transform=c -> deuteranopic(c, 0.5),
                                   lchoices=Float64[65, 70, 75, 80],
                                   cchoices=Float64[0, 50, 60, 70],
                                   hchoices=linspace(0, 330, 24))

oscillatorx1_state_plot = plot(
    layer(x=x_data*Δt, y=noisyObs[1,:], Geom.point, Theme(default_color=getColors[2])),
    layer(x=x_data*Δt, y=trueState[1,:], Geom.line, Theme(default_color=getColors[3])),
    layer(df_fs, x=:x, y=:y, ymin=:ymin, ymax=:ymax, Geom.line, Geom.ribbon),
    Guide.xlabel("Time (seconds)"), Guide.ylabel("x1"),
    Guide.manual_color_key("Colour Key",["Filtered Estimate", "Measurements","True Value "],[getColors[1],getColors[2],getColors[3]]),
    Guide.title("Unscented Kalman Filter (Additive) Example")
    )
display(oscillatorx1_state_plot)

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 Time (seconds) 
 
 
 -30 
 -25 
 -20 
 -15 
 -10 
 -5 
 0 
 5 
 10 
 15 
 20 
 25 
 30 
 35 
 40 
 45 
 50 
 55 
 -25 
 -24 
 -23 
 -22 
 -21 
 -20 
 -19 
 -18 
 -17 
 -16 
 -15 
 -14 
 -13 
 -12 
 -11 
 -10 
 -9 
 -8 
 -7 
 -6 
 -5 
 -4 
 -3 
 -2 
 -1 
 0 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 16 
 17 
 18 
 19 
 20 
 21 
 22 
 23 
 24 
 25 
 26 
 27 
 28 
 29 
 30 
 31 
 32 
 33 
 34 
 35 
 36 
 37 
 38 
 39 
 40 
 41 
 42 
 43 
 44 
 45 
 46 
 47 
 48 
 49 
 50 
 -25 
 0 
 25 
 50 
 -26 
 -24 
 -22 
 -20 
 -18 
 -16 
 -14 
 -12 
 -10 
 -8 
 -6 
 -4 
 -2 
 0 
 2 
 4 
 6 
 8 
 10 
 12 
 14 
 16 
 18 
 20 
 22 
 24 
 26 
 28 
 30 
 32 
 34 
 36 
 38 
 40 
 42 
 44 
 46 
 48 
 50 
 
 
 
 Filtered Estimate 
 Measurements 
 True Value 
 
 
 
 
 
 
 
 Colour Key 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M91.89,48.36 L 91.52 47.22 91.16 46.69 90.8 46.49 90.44 45.93 90.07 44.79 89.71 44.2 89.35 43.93 88.99 42.98 88.62 42.43 88.26 43.12 87.9 42.89 87.54 42.84 87.17 43 86.81 42.98 86.45 43.12 86.09 42.59 85.72 43.09 85.36 43.06 85 44.28 84.64 45.16 84.27 45.57 83.91 46.02 83.55 47.07 83.19 47.65 82.83 48.28 82.46 49.07 82.1 49.86 81.74 49.89 81.38 51.15 81.01 51.53 80.65 52.84 80.29 53.88 79.93 55.25 79.56 55.33 79.2 55.27 78.84 57.15 78.48 57.43 78.11 57.24 77.75 58.22 77.39 59.03 77.03 60.72 76.66 60.75 76.3 60.35 75.94 60.74 75.58 60.52 75.21 60.74 74.85 60.1 74.49 59.93 74.13 59.54 73.76 58.74 73.4 58.7 73.04 57.78 72.68 57.27 72.31 55.85 71.95 54.52 71.59 53.66 71.23 52.98 70.86 51.92 70.5 50.38 70.14 48.1 69.78 46.17 69.41 44.73 69.05 43.76 68.69 42.17 68.33 41.28 67.97 40.26 67.6 39.04 67.24 38.29 66.88 38.04 66.52 37.34 66.15 36.43 65.79 34.83 65.43 35.24 65.07 35.85 64.7 37.08 64.34 37.15 63.98 38.8 63.62 39.96 63.25 39.74 62.89 41.1 62.53 42.09 62.17 43.09 61.8 42.95 61.44 43.12 61.08 43.53 60.72 44.05 60.35 46.11 59.99 48.13 59.63 49.56 59.27 50.06 58.9 52.1 58.54 52.99 58.18 54.31 57.82 56.38 57.45 57.7 57.09 58.61 56.73 59.91 56.37 61.16 56 62.6 55.64 63.43 55.28 64.42 54.92 64.62 54.55 65.51 54.19 65.24 53.83 65.43 53.47 64.29 53.11 64.71 52.74 63.98 52.38 63.61 52.02 62.72 51.66 61.42 51.29 59.6 50.93 58.48 50.57 57.82 50.21 57.07 49.84 55.37 49.48 53.85 49.12 52.99 48.76 51.2 48.39 50.83 48.03 49.09 47.67 47.81 47.31 46.87 46.94 44.74 46.58 42.7 46.22 41.05 45.86 40.03 45.49 37.78 45.13 36.92 44.77 36.16 44.41 35.18 44.04 33.84 43.68 33.56 43.32 33.58 42.96 32.34 42.59 32.52 42.23 32.5 41.87 33.36 41.51 33.47 41.14 33.82 40.78 34.62 40.42 35.1 40.06 35.76 39.69 36.33 39.33 37.37 38.97 39.37 38.61 40.63 38.25 42.87 37.88 44.22 37.52 45.72 37.16 48.17 36.8 50.5 36.43 51.6 36.07 53.87 35.71 55.68 35.35 58.72 34.98 60.57 34.62 62.27 34.26 63.75 33.9 64.17 33.53 65.4 33.17 66.99 32.81 67.87 32.45 69.04 32.08 70.39 31.72 69.95 31.36 70.27 31 70.37 30.63 70.96 30.27 69.9 29.91 69.35 29.55 68.62 29.18 68.6 28.82 67.58 28.46 66.5 28.1 65.39 27.73 62.89 27.37 61.38 27.01 59.48 26.65 58.87 26.28 56.49 25.92 54.39 25.56 51.75 25.2 49.13 24.83 47.38 24.47 46 24.11 44.25 23.75 42.56 23.39 39.54 23.02 38.04 22.66 36.97 22.3 35.5 21.94 34.09 21.57 33.23 21.21 31.85 20.85 32 20.49 34.47 20.12 39.1 19.76 38.4 19.76 18.23 20.12 24.62 20.49 22.47 20.85 21.45 21.21 22.26 21.57 24.33 21.94 25.7 22.3 27.5 22.66 29.27 23.02 30.56 23.39 32.24 23.75 35.38 24.11 37.17 24.47 38.98 24.83 40.42 25.2 42.23 25.56 44.9 25.92 47.58 26.28 49.72 26.65 52.13 27.01 52.77 27.37 54.7 27.73 56.23 28.1 58.74 28.46 59